In [1]:
import pandas as pd
import numpy as np

features_all = pd.read_csv('features_all_weeks.csv')
features_all.drop(columns = ['Unnamed: 0', 'X1', 'X'], inplace = True)
# display(features_all)
print(features_all.info())

train_df = features_all[features_all['week'] != 8]
test_df = features_all[features_all['week'] == 8]

train_df.drop(columns = ['week'], inplace = True)
test_df.drop(columns = ['week'], inplace = True)

print(train_df.info())
print(test_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 274529 entries, 0 to 274528
Data columns (total 28 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   playId                 274529 non-null  int64  
 1   gameId                 274529 non-null  int64  
 2   frameId                274529 non-null  int64  
 3   area_block             274529 non-null  float64
 4   dist_block             274529 non-null  float64
 5   dist_to_qb_block       274529 non-null  float64
 6   x_force_block          274529 non-null  float64
 7   y_force_block          274529 non-null  float64
 8   norm_weighted_x_block  274529 non-null  float64
 9   norm_weighted_y_block  274529 non-null  float64
 10  area_rush              274529 non-null  float64
 11  dist_rush              274529 non-null  float64
 12  dist_to_qb_rush        274529 non-null  float64
 13  x_force_rush           274529 non-null  float64
 14  y_force_rush           274529 non-nu

C:\Users\Harrison\AppData\Local\Temp\ipykernel_24900\281798644.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.drop(columns = ['week'], inplace = True)
C:\Users\Harrison\AppData\Local\Temp\ipykernel_24900\281798644.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df.drop(columns = ['week'], inplace = True)


In [2]:
def df_split_by_frame(df, train = True):
    df_grouped = df.groupby('frameId')
    df_list = []
    for n, g in df_grouped:
        if (n % 5 == 1 and n < 60):
            df_list.append(g)
            display(g)
    return df_list

In [3]:
df_list = df_split_by_frame(train_df)

,playId,gameId,frameId,area_block,dist_block,dist_to_qb_block,x_force_block,y_force_block,norm_weighted_x_block,norm_weighted_y_block,...,net_x_force_bottom,net_y_force_bottom,net_x_force_middle,net_y_force_middle,net_x_force_top,net_y_force_top,hit,hurry,sack,bad_thing
0,100,2021091205,1,79.224000,2.165379,4.273408,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1471.153017,92.498591,116.641363,-99.199424,0,0,0,0
23,100,2021091207,1,147.416000,1.958878,3.882229,11.490406,-15.277501,11.377318,-15.181720,...,-513.875048,108.477916,-1189.433073,127.876699,-385.243873,-164.002623,0,0,0,0
50,100,2021091211,1,260.583000,2.441142,4.757713,5.559925,-1.503573,5.160426,-1.568445,...,-25.522664,-13.342925,-968.328374,-2032.277706,0.000000,0.000000,0,0,0,0
91,1004,2021091212,1,106.137333,2.026799,4.268407,-2.689365,2.690303,-3.023686,3.024741,...,207.205825,50.664607,2499.145937,796.250825,192.076868,42.441751,0,0,0,0
127,1013,2021091300,1,134.465600,2.050543,4.838709,-4.567519,3.496654,-4.488637,3.417429,...,132.691030,26.442032,732.798637,346.000208,68.527886,34.002633,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242275,983,2021102401,1,162.090700,2.060458,4.492955,0.000000,0.000000,0.000000,0.000000,...,75.083456,37.304423,2080.855151,1247.698239,440.883061,92.587228,0,0,0,0
242301,986,2021102404,1,127.692167,2.184162,3.732754,-3.897120,19.215470,-3.103339,16.960249,...,-22.398609,11.104158,1596.256805,409.589665,0.000000,0.000000,1,1,0,1
242327,990,2021102406,1,321.056600,2.098358,2.156077,0.226770,0.812581,-0.067866,1.229622,...,-246.973491,93.274319,-1048.107690,-184.004270,0.454101,-7.576403,0,0,0,0
242351,994,2021102403,1,127.520000,2.003745,4.249218,0.000000,0.000000,0.000000,0.000000,...,-124.187518,13.281486,-1464.515745,-66.628310,-123.694218,17.670393,0,0,0,0


,playId,gameId,frameId,area_block,dist_block,dist_to_qb_block,x_force_block,y_force_block,norm_weighted_x_block,norm_weighted_y_block,...,net_x_force_bottom,net_y_force_bottom,net_x_force_middle,net_y_force_middle,net_x_force_top,net_y_force_top,hit,hurry,sack,bad_thing
19,100,2021091205,6,226.363333,2.128901,4.282523,-231.113671,9.797651,-221.169925,20.664325,...,2416.765891,358.687223,2223.118590,591.848845,953.654875,-511.988403,0,0,0,0
46,100,2021091207,6,75.714400,2.025527,3.839530,140.745824,9.518095,138.385866,9.069584,...,-680.075136,-88.153748,-1356.482885,-377.773558,-1106.405214,-85.004686,0,0,0,0
87,100,2021091211,6,195.386600,2.449239,4.816018,70.720480,-344.857682,76.086896,-362.578943,...,-871.059176,-785.728252,-1555.897003,-4588.216869,0.000000,0.000000,0,0,0,0
123,1004,2021091212,6,303.745033,2.106368,4.217441,-314.963802,80.004319,-309.733931,83.935543,...,1626.897963,382.948595,1279.972418,684.119639,655.290712,-171.179056,0,0,0,0
151,1013,2021091300,6,132.259367,2.049007,4.704041,-121.813321,54.054231,-121.799953,53.322314,...,1625.744583,-76.419204,2734.054264,1179.074022,1212.510490,350.719441,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242297,983,2021102401,6,197.851600,2.169937,4.305740,-286.504145,193.162321,-284.862981,198.159666,...,919.662567,238.737715,1506.091584,1027.880109,1338.823258,-64.138236,0,0,0,0
242323,986,2021102404,6,64.341900,2.323677,3.715062,-115.616447,101.539645,-93.819679,80.450705,...,791.477781,1369.691624,1535.709720,-54.883400,992.156588,-230.839065,1,1,0,1
242347,990,2021102406,6,227.322500,2.181950,2.324980,174.575317,-13.503548,162.678222,-3.039040,...,-1423.148134,-51.126808,-1627.956202,138.225816,-1302.372465,303.826514,0,0,0,0
242380,994,2021102403,6,186.961100,2.152660,4.163767,125.928467,-26.153740,124.352855,-27.469767,...,-2198.711056,151.753210,-2132.456926,-826.825307,-1183.945532,-108.159606,0,0,0,0


,playId,gameId,frameId,area_block,dist_block,dist_to_qb_block,x_force_block,y_force_block,norm_weighted_x_block,norm_weighted_y_block,...,net_x_force_bottom,net_y_force_bottom,net_x_force_middle,net_y_force_middle,net_x_force_top,net_y_force_top,hit,hurry,sack,bad_thing
2,100,2021091205,11,50.359333,2.084440,4.290909,-311.417633,45.599327,-302.820681,58.078608,...,1038.638081,139.671426,1330.320348,462.586594,452.505465,-190.680610,0,0,0,0
25,100,2021091207,11,107.786800,2.021536,3.908861,204.114035,-26.939892,207.182455,-28.628133,...,-130.807840,-31.139148,-985.263988,-253.364402,-739.443772,28.657896,0,0,0,0
52,100,2021091211,11,187.192000,2.292519,4.975004,67.139123,-696.179849,61.437297,-724.165498,...,-462.071524,-862.763815,-1018.284704,-2300.145332,0.000000,0.000000,0,0,0,0
93,1004,2021091212,11,319.357533,2.259058,4.545952,-190.024135,70.757924,-188.360942,70.035424,...,478.712002,64.774926,1197.222005,185.282140,379.893104,8.111234,0,0,0,0
129,1013,2021091300,11,37.066300,2.212953,4.663454,-526.953279,219.635709,-534.152010,214.700388,...,597.805158,53.373437,690.060635,418.373734,1124.639826,299.237236,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242277,983,2021102401,11,308.873167,2.360286,4.151055,-197.011999,152.224964,-194.216158,157.049052,...,525.344438,95.328621,1109.965350,611.458046,656.390484,-0.354306,0,0,0,0
242303,986,2021102404,11,46.664433,2.424454,3.705213,-161.309815,38.661549,-133.219560,74.233574,...,533.455376,-34.964514,2039.397492,1265.516039,951.701081,-220.545448,1,1,0,1
242329,990,2021102406,11,244.589700,2.467071,2.944420,298.170485,32.031034,260.045041,24.742451,...,-762.923428,65.430709,-1017.127697,304.781575,-1161.315367,179.169883,0,0,0,0
242353,994,2021102403,11,589.610500,2.307754,4.186325,413.359330,-49.741947,405.749909,-48.624101,...,-672.601901,76.193060,-2234.015410,-225.920088,-446.417730,-97.562059,0,0,0,0


,playId,gameId,frameId,area_block,dist_block,dist_to_qb_block,x_force_block,y_force_block,norm_weighted_x_block,norm_weighted_y_block,...,net_x_force_bottom,net_y_force_bottom,net_x_force_middle,net_y_force_middle,net_x_force_top,net_y_force_top,hit,hurry,sack,bad_thing
7,100,2021091205,16,39.106167,2.128973,4.209363,-229.912022,58.702865,-228.080989,61.767703,...,999.751678,181.539673,721.187815,264.393652,692.481051,-108.251564,0,0,0,0
30,100,2021091207,16,80.636800,2.026325,4.084134,167.274368,-36.818220,169.126858,-35.194563,...,-742.826205,-22.873014,-1369.653616,-620.984107,-858.012062,-183.918123,0,0,0,0
57,100,2021091211,16,245.808300,2.108303,5.783651,36.697072,-171.095584,34.617514,-185.031140,...,-188.932212,-2128.717686,-793.461039,-1825.674326,0.000000,0.000000,0,0,0,0
98,1004,2021091212,16,365.872733,2.454922,4.956384,-179.018801,59.066455,-175.070195,55.904271,...,986.159388,293.385702,1308.517815,134.811890,1564.467172,190.888742,0,0,0,0
134,1013,2021091300,16,67.430667,2.534239,4.794776,-260.180549,131.121653,-255.666587,123.685401,...,684.173405,84.775158,1561.018823,-189.625418,1969.584275,-257.294483,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242282,983,2021102401,16,262.989333,2.403716,4.056498,-300.552315,126.431231,-305.134119,126.999823,...,98.721342,26.359983,877.695231,333.177573,2508.193296,-497.066847,0,0,0,0
242308,986,2021102404,16,93.060133,2.207566,3.356888,-315.074117,64.721769,-261.772627,113.639158,...,424.748193,87.265550,2090.421836,1631.565542,500.171661,-307.422670,1,1,0,1
242334,990,2021102406,16,248.897900,2.800232,3.546107,199.380285,58.684959,201.660345,54.253992,...,-1277.744755,462.907645,-1442.133380,221.184640,-714.877951,170.150823,0,0,0,0
242358,994,2021102403,16,198.640900,2.259761,4.391623,383.607218,-50.137801,386.677026,-51.483367,...,-432.688249,10.189241,-2245.403923,-62.451700,-1031.810587,-625.338935,0,0,0,0


,playId,gameId,frameId,area_block,dist_block,dist_to_qb_block,x_force_block,y_force_block,norm_weighted_x_block,norm_weighted_y_block,...,net_x_force_bottom,net_y_force_bottom,net_x_force_middle,net_y_force_middle,net_x_force_top,net_y_force_top,hit,hurry,sack,bad_thing
13,100,2021091205,21,34.871667,2.147927,3.593978,-247.612076,63.444753,-247.611861,63.682174,...,1007.913112,646.323643,1244.008922,605.270515,1081.948886,-157.062430,0,0,0,0
36,100,2021091207,21,39.499400,2.214333,3.832495,277.875070,-77.820807,283.013896,-76.148996,...,-1004.638866,510.166129,-2051.728035,175.938102,-506.815086,-288.144924,0,0,0,0
63,100,2021091211,21,18.353600,2.103513,7.996194,100.794143,-410.990922,118.372145,-449.243120,...,-198.167086,-4272.169646,-1451.698661,-789.566154,0.000000,0.000000,0,0,0,0
104,1004,2021091212,21,96.176133,2.539989,4.946178,-276.737470,39.114836,-272.849247,36.968628,...,1213.703227,573.227639,1363.152921,391.232915,1074.043603,-29.920256,0,0,0,0
140,1013,2021091300,21,91.408167,2.813816,4.308558,-445.165558,58.751974,-421.904638,62.554923,...,0.000000,0.000000,2358.125893,1227.789530,1554.278909,-1267.407031,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242288,983,2021102401,21,5.438467,2.211943,3.719584,-382.269399,52.335395,-366.377223,57.593887,...,0.000000,0.000000,2605.791019,-7.407986,1192.471082,-706.954941,0,0,0,0
242314,986,2021102404,21,21.599600,1.804958,2.799449,-254.848205,131.577728,-231.146151,129.783659,...,0.000000,0.000000,3812.531781,2036.642319,793.582263,377.839471,1,1,0,1
242340,990,2021102406,21,284.772500,3.099916,3.686078,300.885905,71.370868,287.633419,69.623145,...,-1036.251941,1940.606770,-1285.400632,450.037598,-1679.749651,340.481876,0,0,0,0
242364,994,2021102403,21,158.464500,2.161148,4.236994,265.253060,-76.071366,261.143539,-74.236240,...,-878.114242,69.030982,-3477.580477,71.862278,-521.226306,-413.859764,0,0,0,0


...

,playId,gameId,frameId,area_block,dist_block,dist_to_qb_block,x_force_block,y_force_block,norm_weighted_x_block,norm_weighted_y_block,...,net_x_force_bottom,net_y_force_bottom,net_x_force_middle,net_y_force_middle,net_x_force_top,net_y_force_top,hit,hurry,sack,bad_thing
79,100,2021091211,36,69.420600,2.731333,17.261649,38.061660,-523.590333,67.796546,-520.898274,...,1967.834531,-273.854057,0.000000,0.000000,385.059967,676.452986,0,0,0,0
120,1004,2021091212,36,250.908367,2.301915,4.024432,-210.566530,34.814212,-201.426761,38.138746,...,-41.632458,-816.198913,2564.359820,-1808.265827,32.414272,16.845051,0,0,0,0
184,1020,2021091210,36,481.246500,2.854926,5.776963,404.388953,70.786087,434.472990,127.970791,...,-815.017119,-120.804559,316.686146,1192.222432,-416.934250,627.360573,0,1,0,1
548,1064,2021091213,36,538.139467,2.615113,2.619856,-220.626764,-7.179108,-235.559332,-9.588840,...,-128.579186,-674.035692,1764.917793,-2701.555346,632.908209,-43.480299,0,1,0,1
611,1075,2021091300,36,0.463600,2.312122,5.259917,49.029671,-5.175992,53.457806,7.946803,...,0.000000,0.000000,-387.033989,1185.418661,0.000000,0.000000,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241604,896,2021102406,36,94.826333,2.269860,3.999681,-364.069481,-111.287334,-337.903889,-154.640809,...,1656.359618,980.946210,-1203.830108,2425.773151,0.000000,0.000000,0,1,0,1
241855,933,2021102500,36,101.618500,1.926882,3.980635,482.156778,-74.230326,445.155638,-106.776560,...,-196.003235,-55.722026,-6294.612607,223.853537,0.000000,0.000000,0,1,0,1
242004,959,2021102401,36,156.838200,2.560341,16.521090,-190.517006,389.104444,-229.009364,369.351606,...,0.000000,0.000000,158.868469,-532.235247,-1469.302969,2064.298166,0,0,0,0
242097,965,2021102405,36,191.267400,2.312172,4.898991,282.040649,-6.653772,269.509634,44.132541,...,-824.340591,-548.215897,-2071.043895,-1167.391976,-113.198023,0.964409,1,0,0,1


,playId,gameId,frameId,area_block,dist_block,dist_to_qb_block,x_force_block,y_force_block,norm_weighted_x_block,norm_weighted_y_block,...,net_x_force_bottom,net_y_force_bottom,net_x_force_middle,net_y_force_middle,net_x_force_top,net_y_force_top,hit,hurry,sack,bad_thing
85,100,2021091211,41,0.000000,2.814631,19.122353,-229.448621,-139.835341,-225.545355,-100.661613,...,952.260869,1179.198636,201.301189,150.264015,683.879006,555.179123,0,0,0,0
617,1075,2021091300,41,55.752800,2.692783,4.404126,25.170114,37.099095,19.942298,48.312786,...,0.000000,0.000000,805.256952,2115.470452,-1777.781183,1480.805087,0,0,0,0
879,1090,2021091207,41,181.083733,5.287117,10.692359,-93.528329,219.357641,-165.680859,271.219581,...,-226.251528,41.076133,-734.988759,2637.913532,-965.910382,2115.591607,0,0,1,1
1053,1122,2021091300,41,126.837800,2.979597,4.813975,329.687974,30.344976,403.000243,61.619221,...,0.000000,0.000000,-4954.053788,934.883005,0.000000,0.000000,0,1,1,1
1328,1153,2021091200,41,294.804300,4.000054,5.011842,239.631521,-220.423761,246.330089,-250.437723,...,-273.288115,-387.829462,-1688.059299,-614.009052,0.000000,0.000000,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241210,833,2021102500,41,74.305500,4.952502,5.299738,-413.233346,120.502981,-394.120944,149.679807,...,655.346905,1051.531947,689.514981,-18.283603,3448.208618,938.753426,0,1,0,1
241530,893,2021102401,41,466.803500,4.099693,10.155252,71.647342,233.226523,83.977123,272.095688,...,0.000000,0.000000,-810.770528,550.116398,430.942906,2815.941506,0,0,0,0
241610,896,2021102406,41,372.273400,2.632926,5.792820,-49.228921,29.358745,72.573963,7.141132,...,600.887241,555.649008,-1125.771734,4257.160778,-632.899386,842.024653,0,1,0,1
241861,933,2021102500,41,79.195200,2.055208,5.510543,545.902216,-29.960211,567.399737,-68.299814,...,-968.443099,-55.796220,-4150.598554,3539.650429,0.000000,0.000000,0,1,0,1


,playId,gameId,frameId,area_block,dist_block,dist_to_qb_block,x_force_block,y_force_block,norm_weighted_x_block,norm_weighted_y_block,...,net_x_force_bottom,net_y_force_bottom,net_x_force_middle,net_y_force_middle,net_x_force_top,net_y_force_top,hit,hurry,sack,bad_thing
884,1090,2021091207,46,695.002033,5.602871,12.962653,56.808130,345.770345,58.393338,422.538418,...,-138.605462,42.243771,-231.204434,1255.999531,-2481.634706,2593.276697,0,0,1,1
1438,1166,2021091209,46,187.098300,4.872376,9.015858,339.610754,-205.562393,406.784469,37.014638,...,-288.979336,-205.568225,730.415808,-1272.942433,619.800475,-893.102496,0,1,0,1
1615,1179,2021091205,46,43.749200,2.664037,11.984557,117.650200,39.284652,118.890225,61.012640,...,0.000000,0.000000,-848.982720,2.575888,-661.591635,1087.875004,0,0,0,0
2467,1271,2021091200,46,41.345100,3.923360,13.191199,-35.882179,-207.238772,43.608650,-194.321112,...,-558.709044,-1458.554268,-38.461200,-811.900588,0.000000,0.000000,0,1,0,1
2626,1293,2021091300,46,83.582467,3.290652,5.085771,-25.544114,-261.233158,-99.004062,-233.228676,...,474.931969,-998.817515,1572.834180,-2214.400214,0.000000,0.000000,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241056,825,2021102406,46,162.151700,2.010005,3.987298,-0.929026,-0.529441,-0.865248,-0.583075,...,0.000000,0.000000,0.787695,-0.433034,10.185024,1.780243,0,0,0,0
241215,833,2021102500,46,94.208667,4.802156,4.907926,-544.549074,138.779212,-465.278539,159.915297,...,-103.467192,834.087451,6.295643,-32.371233,-3020.482284,325.651035,0,1,0,1
241615,896,2021102406,46,235.347733,3.013513,7.822611,-53.552116,425.927951,39.943763,408.673167,...,0.000000,0.000000,-1891.895095,2665.996334,-971.292505,1152.738442,0,1,0,1
241866,933,2021102500,46,174.353600,2.460937,7.107577,550.528557,274.144555,574.153062,294.959327,...,-1089.761417,-874.116320,-942.933881,3744.188336,-77.574343,247.843321,0,1,0,1


,playId,gameId,frameId,area_block,dist_block,dist_to_qb_block,x_force_block,y_force_block,norm_weighted_x_block,norm_weighted_y_block,...,net_x_force_bottom,net_y_force_bottom,net_x_force_middle,net_y_force_middle,net_x_force_top,net_y_force_top,hit,hurry,sack,bad_thing
890,1090,2021091207,51,390.563133,5.870719,15.868975,189.493469,328.111018,220.450456,344.498867,...,0.000000,0.000000,-320.469651,881.838557,-2715.336191,2593.659346,0,0,1,1
1444,1166,2021091209,51,483.673400,4.491062,10.811027,-101.807193,-146.777504,-71.262362,1.680272,...,-980.044954,-1123.571315,624.864614,-581.815104,371.025212,-463.771380,0,1,0,1
1621,1179,2021091205,51,443.072800,3.113980,15.093778,108.810371,82.053466,110.786181,102.913202,...,0.000000,0.000000,-992.706444,232.611850,-952.008056,1943.155649,0,0,0,0
2632,1293,2021091300,51,105.768767,3.487365,4.735133,-197.809578,-379.410948,-262.434190,-265.496544,...,-111.476526,526.772161,2479.029257,-146.769141,0.000000,0.000000,0,1,0,1
2711,1295,2021091209,51,116.548700,5.036272,14.040778,62.928317,53.576108,81.613166,3.606229,...,-53.155695,-737.948032,810.176784,-650.547219,-919.390565,3455.049068,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240779,786,2021102500,51,151.780100,3.230882,3.868192,-33.136091,-276.992356,-7.985126,-559.756771,...,210.262141,-834.622254,1627.852400,-2029.350533,892.867479,-647.089718,0,0,1,1
240907,802,2021102400,51,130.992100,3.146372,7.728235,247.595233,-274.485307,330.225617,-319.433471,...,507.358883,-2194.971186,532.373262,-1782.574563,0.000000,0.000000,0,0,0,0
241062,825,2021102406,51,157.271867,2.007703,3.969613,-0.667465,-0.047369,-0.612381,-0.045048,...,0.000000,0.000000,294.516461,-7.655429,10.080096,2.060948,0,0,0,0
241872,933,2021102500,51,261.519600,3.203904,8.231207,210.315936,362.180685,178.037383,403.567651,...,545.658205,719.138778,-867.312438,2275.677866,-347.165420,1389.367853,0,1,0,1


,playId,gameId,frameId,area_block,dist_block,dist_to_qb_block,x_force_block,y_force_block,norm_weighted_x_block,norm_weighted_y_block,...,net_x_force_bottom,net_y_force_bottom,net_x_force_middle,net_y_force_middle,net_x_force_top,net_y_force_top,hit,hurry,sack,bad_thing
895,1090,2021091207,56,306.644133,6.028691,19.215614,211.804518,290.930673,224.570313,299.703783,...,0.000000,0.000000,-199.452806,333.389968,-2655.842034,3188.281713,0,0,1,1
1449,1166,2021091209,56,60.247500,4.351731,13.108252,-43.214128,-201.875892,4.292546,-218.379034,...,-1162.388724,-1533.680318,824.094725,-849.007694,37.297053,-171.491060,0,1,0,1
1626,1179,2021091205,56,46.751200,3.466095,18.265231,135.374046,142.908328,136.380471,164.272301,...,0.000000,0.000000,-723.589148,443.653546,-937.543320,2604.365899,0,0,0,0
2637,1293,2021091300,56,237.518300,3.956186,5.539864,-321.463269,-142.554329,-403.941604,-122.309996,...,-91.789299,574.161111,1766.373276,3020.750632,0.000000,0.000000,0,1,0,1
2716,1295,2021091209,56,1090.784600,6.048719,15.207814,8.792563,188.525524,14.403653,78.109995,...,398.339970,-1221.055400,1218.606407,-860.349825,3076.665933,-493.624533,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240434,754,2021102410,56,213.138733,2.404379,4.106655,-7.190806,-36.145870,-10.615097,-55.946160,...,0.000000,0.000000,12.964213,222.888674,0.000000,0.000000,0,0,0,0
240784,786,2021102500,56,177.145300,3.250213,3.700244,-205.750809,-240.475629,-127.211231,-253.052048,...,13.808594,1634.737960,1847.148923,-1346.495208,411.820023,-592.111431,0,0,1,1
240912,802,2021102400,56,70.806500,3.341364,9.462514,-38.434395,-362.765797,-52.044010,-427.349997,...,506.646213,-1604.390724,838.911442,-601.369025,0.000000,0.000000,0,0,0,0
241067,825,2021102406,56,145.475233,2.015752,3.865211,-59.473414,-2.751209,-60.947790,-2.878496,...,470.781155,-289.173606,3049.839955,1170.372198,1084.546168,161.238911,0,0,0,0


In [4]:
from sklearn.ensemble import RandomForestClassifier
models = []
for df in df_list:
    X_train = df.drop(columns = ['bad_thing', 'hit', 'hurry', 'sack', 'playId', 'gameId', 'frameId'])
    display(X_train)
    y_train = df['bad_thing']
    rf = RandomForestClassifier(n_estimators=500, oob_score=True, random_state=123456, max_depth = 15)
    rf.fit(X_train, y_train)
    models.append(rf)
    del rf
    

,area_block,dist_block,dist_to_qb_block,x_force_block,y_force_block,norm_weighted_x_block,norm_weighted_y_block,area_rush,dist_rush,dist_to_qb_rush,x_force_rush,y_force_rush,norm_weighted_x_rush,norm_weighted_y_rush,net_x_force_bottom,net_y_force_bottom,net_x_force_middle,net_y_force_middle,net_x_force_top,net_y_force_top
0,79.224000,2.165379,4.273408,0.000000,0.000000,0.000000,0.000000,217.635233,3.941905,7.215590,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1471.153017,92.498591,116.641363,-99.199424
23,147.416000,1.958878,3.882229,11.490406,-15.277501,11.377318,-15.181720,164.844800,4.578860,7.393464,-7.967483,43.416649,-6.734123,41.551634,-513.875048,108.477916,-1189.433073,127.876699,-385.243873,-164.002623
50,260.583000,2.441142,4.757713,5.559925,-1.503573,5.160426,-1.568445,138.509000,2.542495,6.524477,25.561290,-22.612740,24.134830,-22.871755,-25.522664,-13.342925,-968.328374,-2032.277706,0.000000,0.000000
91,106.137333,2.026799,4.268407,-2.689365,2.690303,-3.023686,3.024741,470.301333,3.671248,6.634085,9.421548,-50.203296,11.221523,-58.027434,207.205825,50.664607,2499.145937,796.250825,192.076868,42.441751
127,134.465600,2.050543,4.838709,-4.567519,3.496654,-4.488637,3.417429,153.025533,3.648293,7.264709,-10.973746,7.272435,-9.764310,6.372673,132.691030,26.442032,732.798637,346.000208,68.527886,34.002633
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242275,162.090700,2.060458,4.492955,0.000000,0.000000,0.000000,0.000000,231.963200,3.658547,6.891475,-0.345126,1.957305,-0.287162,1.628578,75.083456,37.304423,2080.855151,1247.698239,440.883061,92.587228
242301,127.692167,2.184162,3.732754,-3.897120,19.215470,-3.103339,16.960249,60.306600,3.324317,6.404702,10.994239,-5.088897,10.293713,-4.577981,-22.398609,11.104158,1596.256805,409.589665,0.000000,0.000000
242327,321.056600,2.098358,2.156077,0.226770,0.812581,-0.067866,1.229622,249.258800,3.987996,4.786245,59.792388,-111.038432,58.795564,-69.883071,-246.973491,93.274319,-1048.107690,-184.004270,0.454101,-7.576403
242351,127.520000,2.003745,4.249218,0.000000,0.000000,0.000000,0.000000,273.353500,3.695239,6.818202,-1.940854,0.477905,-2.421882,1.403031,-124.187518,13.281486,-1464.515745,-66.628310,-123.694218,17.670393


,area_block,dist_block,dist_to_qb_block,x_force_block,y_force_block,norm_weighted_x_block,norm_weighted_y_block,area_rush,dist_rush,dist_to_qb_rush,x_force_rush,y_force_rush,norm_weighted_x_rush,norm_weighted_y_rush,net_x_force_bottom,net_y_force_bottom,net_x_force_middle,net_y_force_middle,net_x_force_top,net_y_force_top
19,226.363333,2.128901,4.282523,-231.113671,9.797651,-221.169925,20.664325,238.432200,3.809630,7.067729,-144.075354,-18.563033,-167.337886,-9.809088,2416.765891,358.687223,2223.118590,591.848845,953.654875,-511.988403
46,75.714400,2.025527,3.839530,140.745824,9.518095,138.385866,9.069584,0.000000,4.356989,6.892317,382.655170,36.451731,362.679110,40.744670,-680.075136,-88.153748,-1356.482885,-377.773558,-1106.405214,-85.004686
87,195.386600,2.449239,4.816018,70.720480,-344.857682,76.086896,-362.578943,266.684600,2.560221,6.352579,130.357471,-72.879822,144.251528,-71.290729,-871.059176,-785.728252,-1555.897003,-4588.216869,0.000000,0.000000
123,303.745033,2.106368,4.217441,-314.963802,80.004319,-309.733931,83.935543,442.878167,3.632504,6.310562,-330.902410,122.333890,-351.015454,133.999143,1626.897963,382.948595,1279.972418,684.119639,655.290712,-171.179056
151,132.259367,2.049007,4.704041,-121.813321,54.054231,-121.799953,53.322314,102.216000,3.630008,6.895700,-81.237743,34.043428,-79.599823,34.687759,1625.744583,-76.419204,2734.054264,1179.074022,1212.510490,350.719441
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242297,197.851600,2.169937,4.305740,-286.504145,193.162321,-284.862981,198.159666,556.466933,3.604609,6.455761,-291.075207,102.944554,-289.821926,108.222092,919.662567,238.737715,1506.091584,1027.880109,1338.823258,-64.138236
242323,64.341900,2.323677,3.715062,-115.616447,101.539645,-93.819679,80.450705,8.504467,3.241785,6.054896,-220.039881,-47.136013,-255.438947,-54.951367,791.477781,1369.691624,1535.709720,-54.883400,992.156588,-230.839065
242347,227.322500,2.181950,2.324980,174.575317,-13.503548,162.678222,-3.039040,262.503800,3.941625,4.824311,105.437630,-7.697185,92.357385,-23.335479,-1423.148134,-51.126808,-1627.956202,138.225816,-1302.372465,303.826514
242380,186.961100,2.152660,4.163767,125.928467,-26.153740,124.352855,-27.469767,525.843500,3.578477,6.494121,216.551028,19.018428,236.533022,18.742679,-2198.711056,151.753210,-2132.456926,-826.825307,-1183.945532,-108.159606


,area_block,dist_block,dist_to_qb_block,x_force_block,y_force_block,norm_weighted_x_block,norm_weighted_y_block,area_rush,dist_rush,dist_to_qb_rush,x_force_rush,y_force_rush,norm_weighted_x_rush,norm_weighted_y_rush,net_x_force_bottom,net_y_force_bottom,net_x_force_middle,net_y_force_middle,net_x_force_top,net_y_force_top
2,50.359333,2.084440,4.290909,-311.417633,45.599327,-302.820681,58.078608,230.288000,3.501304,6.150107,-1345.483725,70.183668,-1315.285185,98.733910,1038.638081,139.671426,1330.320348,462.586594,452.505465,-190.680610
25,107.786800,2.021536,3.908861,204.114035,-26.939892,207.182455,-28.628133,266.712000,4.209722,6.407851,535.394092,-47.080676,526.219528,-36.071138,-130.807840,-31.139148,-985.263988,-253.364402,-739.443772,28.657896
52,187.192000,2.292519,4.975004,67.139123,-696.179849,61.437297,-724.165498,321.494600,2.566646,6.067153,492.928413,-481.379288,490.375074,-478.551600,-462.071524,-862.763815,-1018.284704,-2300.145332,0.000000,0.000000
93,319.357533,2.259058,4.545952,-190.024135,70.757924,-188.360942,70.035424,112.587867,3.600125,6.029223,-653.010090,135.524885,-621.071519,139.994716,478.712002,64.774926,1197.222005,185.282140,379.893104,8.111234
129,37.066300,2.212953,4.663454,-526.953279,219.635709,-534.152010,214.700388,184.963333,3.580642,6.161498,-734.385760,88.798919,-736.221224,96.861508,597.805158,53.373437,690.060635,418.373734,1124.639826,299.237236
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242277,308.873167,2.360286,4.151055,-197.011999,152.224964,-194.216158,157.049052,471.541500,3.514460,5.639036,-694.879376,110.338686,-668.675816,116.474702,525.344438,95.328621,1109.965350,611.458046,656.390484,-0.354306
242303,46.664433,2.424454,3.705213,-161.309815,38.661549,-133.219560,74.233574,93.812233,2.917205,5.278658,-587.871333,212.999967,-582.336253,170.841452,533.455376,-34.964514,2039.397492,1265.516039,951.701081,-220.545448
242329,244.589700,2.467071,2.944420,298.170485,32.031034,260.045041,24.742451,266.718400,3.915335,4.895117,715.656118,57.692451,667.186394,56.132620,-762.923428,65.430709,-1017.127697,304.781575,-1161.315367,179.169883
242353,589.610500,2.307754,4.186325,413.359330,-49.741947,405.749909,-48.624101,241.767600,3.220855,5.820879,689.663373,-106.904464,697.074770,-136.845594,-672.601901,76.193060,-2234.015410,-225.920088,-446.417730,-97.562059


,area_block,dist_block,dist_to_qb_block,x_force_block,y_force_block,norm_weighted_x_block,norm_weighted_y_block,area_rush,dist_rush,dist_to_qb_rush,x_force_rush,y_force_rush,norm_weighted_x_rush,norm_weighted_y_rush,net_x_force_bottom,net_y_force_bottom,net_x_force_middle,net_y_force_middle,net_x_force_top,net_y_force_top
7,39.106167,2.128973,4.209363,-229.912022,58.702865,-228.080989,61.767703,89.460467,3.465230,5.079902,-557.301249,39.354358,-552.060070,48.063540,999.751678,181.539673,721.187815,264.393652,692.481051,-108.251564
30,80.636800,2.026325,4.084134,167.274368,-36.818220,169.126858,-35.194563,211.083000,4.606236,6.075538,250.469855,57.098574,287.247807,68.291311,-742.826205,-22.873014,-1369.653616,-620.984107,-858.012062,-183.918123
57,245.808300,2.108303,5.783651,36.697072,-171.095584,34.617514,-185.031140,241.620600,2.630863,6.326985,322.323151,-514.172327,355.356961,-562.624697,-188.932212,-2128.717686,-793.461039,-1825.674326,0.000000,0.000000
98,365.872733,2.454922,4.956384,-179.018801,59.066455,-175.070195,55.904271,80.616033,3.635777,5.983106,-290.183276,-9.290994,-295.993023,-3.785225,986.159388,293.385702,1308.517815,134.811890,1564.467172,190.888742
134,67.430667,2.534239,4.794776,-260.180549,131.121653,-255.666587,123.685401,149.351200,3.644064,5.549093,-277.900720,28.844035,-290.808273,33.879752,684.173405,84.775158,1561.018823,-189.625418,1969.584275,-257.294483
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242282,262.989333,2.403716,4.056498,-300.552315,126.431231,-305.134119,126.999823,245.751467,3.287590,4.910292,-197.234679,18.569045,-197.434034,21.165067,98.721342,26.359983,877.695231,333.177573,2508.193296,-497.066847
242308,93.060133,2.207566,3.356888,-315.074117,64.721769,-261.772627,113.639158,62.813033,2.420951,4.174388,-408.527312,155.418857,-402.555512,159.352111,424.748193,87.265550,2090.421836,1631.565542,500.171661,-307.422670
242334,248.897900,2.800232,3.546107,199.380285,58.684959,201.660345,54.253992,162.112300,3.990940,4.792709,486.116285,63.989234,438.546794,65.390192,-1277.744755,462.907645,-1442.133380,221.184640,-714.877951,170.150823
242358,198.640900,2.259761,4.391623,383.607218,-50.137801,386.677026,-51.483367,563.420900,2.814636,5.418243,286.999790,0.679925,283.005104,-3.822850,-432.688249,10.189241,-2245.403923,-62.451700,-1031.810587,-625.338935


,area_block,dist_block,dist_to_qb_block,x_force_block,y_force_block,norm_weighted_x_block,norm_weighted_y_block,area_rush,dist_rush,dist_to_qb_rush,x_force_rush,y_force_rush,norm_weighted_x_rush,norm_weighted_y_rush,net_x_force_bottom,net_y_force_bottom,net_x_force_middle,net_y_force_middle,net_x_force_top,net_y_force_top
13,34.871667,2.147927,3.593978,-247.612076,63.444753,-247.611861,63.682174,284.373167,3.275023,4.018641,-391.786727,6.819328,-389.438239,8.897273,1007.913112,646.323643,1244.008922,605.270515,1081.948886,-157.062430
36,39.499400,2.214333,3.832495,277.875070,-77.820807,283.013896,-76.148996,0.000000,4.609412,5.236741,401.671205,-7.182550,393.919428,-27.218539,-1004.638866,510.166129,-2051.728035,175.938102,-506.815086,-288.144924
63,18.353600,2.103513,7.996194,100.794143,-410.990922,118.372145,-449.243120,224.081000,3.035057,7.746319,178.571022,-410.638860,221.085943,-394.501796,-198.167086,-4272.169646,-1451.698661,-789.566154,0.000000,0.000000
104,96.176133,2.539989,4.946178,-276.737470,39.114836,-272.849247,36.968628,287.639567,3.438203,5.653219,-618.864263,105.878028,-621.321265,110.641503,1213.703227,573.227639,1363.152921,391.232915,1074.043603,-29.920256
140,91.408167,2.813816,4.308558,-445.165558,58.751974,-421.904638,62.554923,148.230733,3.635477,4.666806,-497.237182,-163.976149,-456.430978,-116.871570,0.000000,0.000000,2358.125893,1227.789530,1554.278909,-1267.407031
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242288,5.438467,2.211943,3.719584,-382.269399,52.335395,-366.377223,57.593887,57.219033,2.847241,4.188625,-393.315708,-99.801564,-366.373209,-86.632925,0.000000,0.000000,2605.791019,-7.407986,1192.471082,-706.954941
242314,21.599600,1.804958,2.799449,-254.848205,131.577728,-231.146151,129.783659,31.721533,1.888033,3.097693,-371.563122,155.485507,-389.429898,155.618329,0.000000,0.000000,3812.531781,2036.642319,793.582263,377.839471
242340,284.772500,3.099916,3.686078,300.885905,71.370868,287.633419,69.623145,284.730200,4.084379,4.563915,482.581664,124.347085,488.147692,128.438755,-1036.251941,1940.606770,-1285.400632,450.037598,-1679.749651,340.481876
242364,158.464500,2.161148,4.236994,265.253060,-76.071366,261.143539,-74.236240,5.868800,2.659681,4.938723,476.727500,-59.448969,493.484158,-53.172934,-878.114242,69.030982,-3477.580477,71.862278,-521.226306,-413.859764


...

,area_block,dist_block,dist_to_qb_block,x_force_block,y_force_block,norm_weighted_x_block,norm_weighted_y_block,area_rush,dist_rush,dist_to_qb_rush,x_force_rush,y_force_rush,norm_weighted_x_rush,norm_weighted_y_rush,net_x_force_bottom,net_y_force_bottom,net_x_force_middle,net_y_force_middle,net_x_force_top,net_y_force_top
79,69.420600,2.731333,17.261649,38.061660,-523.590333,67.796546,-520.898274,586.245200,4.913752,13.261687,-332.759551,374.504719,-290.609321,688.761915,1967.834531,-273.854057,0.000000,0.000000,385.059967,676.452986
120,250.908367,2.301915,4.024432,-210.566530,34.814212,-201.426761,38.138746,0.974867,3.159813,4.435191,-358.310876,127.615286,-360.512930,123.598999,-41.632458,-816.198913,2564.359820,-1808.265827,32.414272,16.845051
184,481.246500,2.854926,5.776963,404.388953,70.786087,434.472990,127.970791,782.904000,3.866345,6.349392,56.478153,322.746683,109.459034,425.965601,-815.017119,-120.804559,316.686146,1192.222432,-416.934250,627.360573
548,538.139467,2.615113,2.619856,-220.626764,-7.179108,-235.559332,-9.588840,329.930667,2.526876,2.765977,-354.304952,-234.051283,-345.631472,-247.870598,-128.579186,-674.035692,1764.917793,-2701.555346,632.908209,-43.480299
611,0.463600,2.312122,5.259917,49.029671,-5.175992,53.457806,7.946803,23.580300,3.696633,5.876758,65.308175,-93.578069,64.353983,-67.624218,0.000000,0.000000,-387.033989,1185.418661,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241604,94.826333,2.269860,3.999681,-364.069481,-111.287334,-337.903889,-154.640809,221.465433,3.019423,4.386878,-605.358464,176.803490,-533.109217,119.448825,1656.359618,980.946210,-1203.830108,2425.773151,0.000000,0.000000
241855,101.618500,1.926882,3.980635,482.156778,-74.230326,445.155638,-106.776560,176.124600,1.539729,3.877651,619.054138,120.073281,604.268876,32.481533,-196.003235,-55.722026,-6294.612607,223.853537,0.000000,0.000000
242004,156.838200,2.560341,16.521090,-190.517006,389.104444,-229.009364,369.351606,559.010567,4.491812,17.684228,25.562124,293.904735,-29.167544,169.444420,0.000000,0.000000,158.868469,-532.235247,-1469.302969,2064.298166
242097,191.267400,2.312172,4.898991,282.040649,-6.653772,269.509634,44.132541,158.605200,3.086308,4.400224,633.800473,-150.627619,652.201241,-131.119225,-824.340591,-548.215897,-2071.043895,-1167.391976,-113.198023,0.964409


,area_block,dist_block,dist_to_qb_block,x_force_block,y_force_block,norm_weighted_x_block,norm_weighted_y_block,area_rush,dist_rush,dist_to_qb_rush,x_force_rush,y_force_rush,norm_weighted_x_rush,norm_weighted_y_rush,net_x_force_bottom,net_y_force_bottom,net_x_force_middle,net_y_force_middle,net_x_force_top,net_y_force_top
85,0.000000,2.814631,19.122353,-229.448621,-139.835341,-225.545355,-100.661613,896.778900,5.277910,13.849124,-336.019888,189.111302,-353.759643,412.346111,952.260869,1179.198636,201.301189,150.264015,683.879006,555.179123
617,55.752800,2.692783,4.404126,25.170114,37.099095,19.942298,48.312786,80.355500,4.056845,5.181349,65.295872,249.980775,107.543438,311.430866,0.000000,0.000000,805.256952,2115.470452,-1777.781183,1480.805087
879,181.083733,5.287117,10.692359,-93.528329,219.357641,-165.680859,271.219581,126.064733,3.501222,10.256812,186.662604,753.620477,144.011645,792.142108,-226.251528,41.076133,-734.988759,2637.913532,-965.910382,2115.591607
1053,126.837800,2.979597,4.813975,329.687974,30.344976,403.000243,61.619221,164.052600,3.190578,3.715233,723.409089,175.663698,704.150154,-181.315907,0.000000,0.000000,-4954.053788,934.883005,0.000000,0.000000
1328,294.804300,4.000054,5.011842,239.631521,-220.423761,246.330089,-250.437723,287.334000,3.923505,4.785630,321.696441,-446.478013,341.650440,-588.492283,-273.288115,-387.829462,-1688.059299,-614.009052,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241210,74.305500,4.952502,5.299738,-413.233346,120.502981,-394.120944,149.679807,401.870000,4.645570,5.356043,-824.906893,66.837942,-839.979671,8.570986,655.346905,1051.531947,689.514981,-18.283603,3448.208618,938.753426
241530,466.803500,4.099693,10.155252,71.647342,233.226523,83.977123,272.095688,264.216600,5.612761,10.658959,47.818808,257.750731,34.907347,309.455524,0.000000,0.000000,-810.770528,550.116398,430.942906,2815.941506
241610,372.273400,2.632926,5.792820,-49.228921,29.358745,72.573963,7.141132,251.293067,3.090484,5.215773,-39.289061,807.641719,-11.296816,775.737587,600.887241,555.649008,-1125.771734,4257.160778,-632.899386,842.024653
241861,79.195200,2.055208,5.510543,545.902216,-29.960211,567.399737,-68.299814,341.011800,2.228764,5.388209,643.040492,69.578413,696.870877,167.112921,-968.443099,-55.796220,-4150.598554,3539.650429,0.000000,0.000000


,area_block,dist_block,dist_to_qb_block,x_force_block,y_force_block,norm_weighted_x_block,norm_weighted_y_block,area_rush,dist_rush,dist_to_qb_rush,x_force_rush,y_force_rush,norm_weighted_x_rush,norm_weighted_y_rush,net_x_force_bottom,net_y_force_bottom,net_x_force_middle,net_y_force_middle,net_x_force_top,net_y_force_top
884,695.002033,5.602871,12.962653,56.808130,345.770345,58.393338,422.538418,139.519333,3.417641,12.084202,382.373661,593.547070,411.611369,642.228029,-138.605462,42.243771,-231.204434,1255.999531,-2481.634706,2593.276697
1438,187.098300,4.872376,9.015858,339.610754,-205.562393,406.784469,37.014638,578.804700,6.304661,8.988150,141.830535,-54.117562,225.596032,112.676951,-288.979336,-205.568225,730.415808,-1272.942433,619.800475,-893.102496
1615,43.749200,2.664037,11.984557,117.650200,39.284652,118.890225,61.012640,8.401900,5.328961,8.498128,203.229115,103.150137,183.285454,175.394225,0.000000,0.000000,-848.982720,2.575888,-661.591635,1087.875004
2467,41.345100,3.923360,13.191199,-35.882179,-207.238772,43.608650,-194.321112,123.838900,6.164984,8.970062,-115.545290,-286.203245,160.283087,-360.226039,-558.709044,-1458.554268,-38.461200,-811.900588,0.000000,0.000000
2626,83.582467,3.290652,5.085771,-25.544114,-261.233158,-99.004062,-233.228676,9.247033,2.857296,3.407386,-246.166352,-338.240186,-236.668871,-305.800696,474.931969,-998.817515,1572.834180,-2214.400214,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241056,162.151700,2.010005,3.987298,-0.929026,-0.529441,-0.865248,-0.583075,112.975000,3.171175,6.467714,-5.639725,18.800859,-6.035755,19.993888,0.000000,0.000000,0.787695,-0.433034,10.185024,1.780243
241215,94.208667,4.802156,4.907926,-544.549074,138.779212,-465.278539,159.915297,20.373667,4.194796,4.709034,-245.306786,250.136810,-150.669923,38.329757,-103.467192,834.087451,6.295643,-32.371233,-3020.482284,325.651035
241615,235.347733,3.013513,7.822611,-53.552116,425.927951,39.943763,408.673167,364.931400,3.172772,5.776400,369.774266,774.816553,389.330667,748.262378,0.000000,0.000000,-1891.895095,2665.996334,-971.292505,1152.738442
241866,174.353600,2.460937,7.107577,550.528557,274.144555,574.153062,294.959327,535.791000,3.324346,6.979047,363.174173,367.797284,342.322584,456.489008,-1089.761417,-874.116320,-942.933881,3744.188336,-77.574343,247.843321


,area_block,dist_block,dist_to_qb_block,x_force_block,y_force_block,norm_weighted_x_block,norm_weighted_y_block,area_rush,dist_rush,dist_to_qb_rush,x_force_rush,y_force_rush,norm_weighted_x_rush,norm_weighted_y_rush,net_x_force_bottom,net_y_force_bottom,net_x_force_middle,net_y_force_middle,net_x_force_top,net_y_force_top
890,390.563133,5.870719,15.868975,189.493469,328.111018,220.450456,344.498867,2.659467,2.990800,14.206306,381.247616,398.685707,410.892048,409.701643,0.000000,0.000000,-320.469651,881.838557,-2715.336191,2593.659346
1444,483.673400,4.491062,10.811027,-101.807193,-146.777504,-71.262362,1.680272,625.423800,6.207035,10.173144,-112.598465,-372.737031,-41.305505,-461.009142,-980.044954,-1123.571315,624.864614,-581.815104,371.025212,-463.771380
1621,443.072800,3.113980,15.093778,108.810371,82.053466,110.786181,102.913202,931.686700,6.027680,10.004538,241.630686,170.045849,192.213526,213.374786,0.000000,0.000000,-992.706444,232.611850,-952.008056,1943.155649
2632,105.768767,3.487365,4.735133,-197.809578,-379.410948,-262.434190,-265.496544,205.982400,3.230901,3.660066,-264.679595,-329.040741,-467.398450,-384.381657,-111.476526,526.772161,2479.029257,-146.769141,0.000000,0.000000
2711,116.548700,5.036272,14.040778,62.928317,53.576108,81.613166,3.606229,418.256000,6.493760,13.134151,257.189196,110.193546,605.050819,-98.305835,-53.155695,-737.948032,810.176784,-650.547219,-919.390565,3455.049068
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240779,151.780100,3.230882,3.868192,-33.136091,-276.992356,-7.985126,-559.756771,54.892000,3.449475,3.411517,-571.229762,-241.680287,-766.382269,109.846291,210.262141,-834.622254,1627.852400,-2029.350533,892.867479,-647.089718
240907,130.992100,3.146372,7.728235,247.595233,-274.485307,330.225617,-319.433471,176.034900,3.917570,7.040637,-76.098930,-415.738798,-138.080563,-475.261627,507.358883,-2194.971186,532.373262,-1782.574563,0.000000,0.000000
241062,157.271867,2.007703,3.969613,-0.667465,-0.047369,-0.612381,-0.045048,234.630733,3.158691,6.453580,-1.908849,0.396013,-2.006346,0.856746,0.000000,0.000000,294.516461,-7.655429,10.080096,2.060948
241872,261.519600,3.203904,8.231207,210.315936,362.180685,178.037383,403.567651,414.534500,4.304562,7.782707,169.674832,188.966248,115.487836,257.110687,545.658205,719.138778,-867.312438,2275.677866,-347.165420,1389.367853


,area_block,dist_block,dist_to_qb_block,x_force_block,y_force_block,norm_weighted_x_block,norm_weighted_y_block,area_rush,dist_rush,dist_to_qb_rush,x_force_rush,y_force_rush,norm_weighted_x_rush,norm_weighted_y_rush,net_x_force_bottom,net_y_force_bottom,net_x_force_middle,net_y_force_middle,net_x_force_top,net_y_force_top
895,306.644133,6.028691,19.215614,211.804518,290.930673,224.570313,299.703783,52.667267,2.489481,16.248127,388.293649,359.745749,400.055561,360.502667,0.000000,0.000000,-199.452806,333.389968,-2655.842034,3188.281713
1449,60.247500,4.351731,13.108252,-43.214128,-201.875892,4.292546,-218.379034,796.932800,6.680242,11.520541,60.859948,-239.475688,95.813720,-325.892956,-1162.388724,-1533.680318,824.094725,-849.007694,37.297053,-171.491060
1626,46.751200,3.466095,18.265231,135.374046,142.908328,136.380471,164.272301,373.197100,6.683995,11.414872,316.961212,365.306378,259.977912,412.620949,0.000000,0.000000,-723.589148,443.653546,-937.543320,2604.365899
2637,237.518300,3.956186,5.539864,-321.463269,-142.554329,-403.941604,-122.309996,275.871400,3.412276,4.688878,-190.059084,273.193610,-362.402427,301.027118,-91.789299,574.161111,1766.373276,3020.750632,0.000000,0.000000
2716,1090.784600,6.048719,15.207814,8.792563,188.525524,14.403653,78.109995,564.710100,7.012860,14.647876,27.403630,233.850064,125.156093,-148.922716,398.339970,-1221.055400,1218.606407,-860.349825,3076.665933,-493.624533
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240434,213.138733,2.404379,4.106655,-7.190806,-36.145870,-10.615097,-55.946160,191.115533,2.023222,6.856334,-50.311899,-52.938724,-48.361725,-40.743911,0.000000,0.000000,12.964213,222.888674,0.000000,0.000000
240784,177.145300,3.250213,3.700244,-205.750809,-240.475629,-127.211231,-253.052048,349.828200,3.009183,2.660710,-425.557018,-577.171087,-536.113914,-482.414748,13.808594,1634.737960,1847.148923,-1346.495208,411.820023,-592.111431
240912,70.806500,3.341364,9.462514,-38.434395,-362.765797,-52.044010,-427.349997,133.773900,4.455290,8.157882,-202.281498,-450.237747,-176.932686,-508.599315,506.646213,-1604.390724,838.911442,-601.369025,0.000000,0.000000
241067,145.475233,2.015752,3.865211,-59.473414,-2.751209,-60.947790,-2.878496,221.165800,3.159967,6.344545,-1.807370,2.040391,-1.714697,2.683336,470.781155,-289.173606,3049.839955,1170.372198,1084.546168,161.238911


In [5]:
# do evaluation
# display(test_df)
test_grouped = test_df.groupby('frameId')
test_grouped_list = []
for n, g in test_grouped:
    if (n % 5 == 1 and n < 60):
        test_grouped_list.append(g)
        # display(g)

test_by_frame_list = []
for df in test_grouped_list:
    test_y = df['bad_thing']
    test_x = df.drop(columns = ['bad_thing', 'hit', 'hurry', 'sack', 'playId', 'gameId', 'frameId'])
    test_by_frame_list.append((test_x, test_y))
    display(test_x)

,area_block,dist_block,dist_to_qb_block,x_force_block,y_force_block,norm_weighted_x_block,norm_weighted_y_block,area_rush,dist_rush,dist_to_qb_rush,x_force_rush,y_force_rush,norm_weighted_x_rush,norm_weighted_y_rush,net_x_force_bottom,net_y_force_bottom,net_x_force_middle,net_y_force_middle,net_x_force_top,net_y_force_top
242418,164.810800,1.855897,4.506846,-4.582069,-4.176678,-4.285960,-3.906767,282.679867,3.507132,7.022569,-12.612916,6.398715,-10.856834,5.597061,-47.242353,33.214788,1345.392584,1.103788,48.631530,15.361462
242453,42.361800,1.907994,4.023396,0.000000,0.000000,0.000000,0.000000,248.679900,3.220757,6.617986,89.634654,46.979456,63.758821,33.417374,-809.180292,363.584606,-1981.270453,-372.821840,0.000000,0.000000
242470,77.264800,1.813165,4.158328,0.000000,0.000000,0.000000,0.000000,218.922900,3.886763,6.708924,-0.504979,-1.844628,-0.434782,-1.588205,-790.060794,-26.346911,-1768.423068,-504.038368,-209.818097,-63.787611
242502,387.524300,1.842362,4.019035,-0.265792,0.077191,-0.261450,-0.010776,301.716100,3.949395,6.829883,25.017271,11.718791,20.763470,11.438514,-148.942519,32.747161,-2116.658117,-701.778887,-84.680328,-19.736886
242531,67.536400,2.493357,2.738903,-0.179453,0.388987,0.066669,0.511709,152.994000,3.278515,4.132863,4.864940,32.918368,6.625845,23.771627,-40.794260,28.807549,-122.389422,1239.313245,28.205362,6.912243
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274393,84.276800,2.004919,4.342458,0.000000,0.000000,0.000000,0.000000,643.549900,3.729955,6.822678,-4.942254,-5.904585,-5.417162,-6.471966,-178.709499,31.704346,-1843.768376,-1033.675940,-562.502913,-179.734010
274417,174.401667,3.089274,3.247840,0.000000,0.000000,0.000000,0.000000,99.408133,3.371638,4.338689,-1.326556,-1.500728,-0.151205,-1.267744,349.924302,729.339807,-226.036999,-1852.872380,24.053982,-133.873069
274449,104.045100,2.450469,2.524039,-0.325991,-0.948898,-0.382414,-1.113135,577.022100,3.518140,4.338452,3.894831,1.331594,3.837430,1.556513,2.395367,1.350821,0.238291,1362.293586,-36.547010,-32.887893
274490,78.689267,1.901695,4.208309,-2.398763,4.496215,-2.337286,5.148423,328.460600,3.590243,6.737212,-137.514510,-51.157388,-156.650000,-57.076173,2.116436,-12.674510,1521.156302,-248.431467,371.082770,-12.699117


,area_block,dist_block,dist_to_qb_block,x_force_block,y_force_block,norm_weighted_x_block,norm_weighted_y_block,area_rush,dist_rush,dist_to_qb_rush,x_force_rush,y_force_rush,norm_weighted_x_rush,norm_weighted_y_rush,net_x_force_bottom,net_y_force_bottom,net_x_force_middle,net_y_force_middle,net_x_force_top,net_y_force_top
242449,108.451167,1.926230,4.364251,-191.127373,31.356279,-192.380365,35.941610,587.028333,3.444205,6.657633,-97.786230,-26.775335,-109.763502,-32.788974,1425.915905,-104.223884,2431.603492,-617.516377,1077.749789,52.269197
242466,159.855800,2.344967,4.077566,311.600540,-32.641552,308.973464,-45.572682,273.532000,2.941285,5.925896,308.112047,38.492590,302.010192,19.689193,-1496.072608,53.045422,-1603.190804,-579.686485,-272.531220,398.767779
242498,79.311000,2.011772,4.119389,213.386391,-96.920079,213.963178,-94.321337,611.543800,3.877264,6.259896,425.342504,-27.393239,408.013965,-21.267570,-1226.875659,-147.263801,-1782.022002,-34.348454,-2020.077116,532.257465
242527,0.000000,1.902635,3.988485,238.896638,-102.978128,250.364561,-112.331402,1154.075600,3.825638,6.506549,288.949431,-43.469571,326.211614,-54.459784,-1300.350815,66.838923,-1357.090193,-1162.381402,-1502.843907,639.530914
242570,142.908100,2.538333,2.843397,8.040942,189.325897,13.506378,181.897970,257.082400,3.261732,4.088607,21.683180,34.769409,27.202951,41.741681,-93.988470,1332.276911,-640.874293,1756.828850,-530.294874,624.158237
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274413,82.191500,2.241276,4.249923,320.414425,-111.143283,314.305585,-96.927483,193.140200,3.677527,6.356680,245.727182,-156.497319,234.054954,-170.321811,-821.180906,-303.065903,-2254.664149,-1356.290791,-1286.738097,-1.455441
274445,66.731167,3.015968,3.288436,96.481726,-174.878719,116.460312,-201.579331,427.318667,2.886441,4.245532,-183.959004,28.324830,-167.402053,-11.020440,1340.674565,417.868288,-799.753543,-2117.448964,-10.682949,-1400.190875
274486,54.037800,2.415000,2.646323,-35.591349,185.344095,-56.870451,246.944549,563.354400,3.462502,4.324039,61.865385,54.672970,75.912610,69.726496,-343.866871,654.969988,-272.681589,4864.975677,-884.697300,-117.128570
274505,56.716667,2.010086,4.037657,-256.836366,-39.454787,-257.207870,-37.964512,459.450833,3.630930,6.371884,-152.543423,-19.132786,-151.703823,-18.569186,1576.063262,-240.195140,1590.654465,-81.424335,1512.903281,102.812269


,area_block,dist_block,dist_to_qb_block,x_force_block,y_force_block,norm_weighted_x_block,norm_weighted_y_block,area_rush,dist_rush,dist_to_qb_rush,x_force_rush,y_force_rush,norm_weighted_x_rush,norm_weighted_y_rush,net_x_force_bottom,net_y_force_bottom,net_x_force_middle,net_y_force_middle,net_x_force_top,net_y_force_top
242420,415.171600,2.185082,4.508423,-399.679920,-28.359728,-398.566387,-26.494518,579.407067,3.319252,6.096730,-734.217412,-131.918107,-741.097719,-143.008462,595.102193,-122.193162,899.940514,-327.855547,449.210687,-124.307413
242455,258.679900,3.004032,4.451250,269.557152,-57.236612,274.206304,-79.877164,228.348700,2.879006,5.456168,506.002186,39.577374,517.374904,29.588785,-949.293192,-101.699286,-1494.400504,-106.110714,-278.733481,316.606605
242472,161.213800,2.408238,4.193858,467.933322,103.637178,457.794862,106.310112,122.859000,3.684867,5.525460,672.327020,-41.885220,653.873973,-40.824598,-931.879411,135.161590,-1086.605350,-172.384958,-682.712560,189.844707
242504,161.921300,2.142659,4.177105,337.459801,-90.878544,315.024756,-94.671894,396.072100,3.732544,6.048802,618.246497,-0.404827,599.167406,-31.259853,-400.906022,-41.782155,-1297.580519,-360.852425,-620.082067,257.783200
242533,416.491100,2.601345,3.443139,-1.448842,455.968480,-2.261444,404.651895,81.511200,3.199005,4.402561,318.462688,244.363277,322.304458,252.612818,-95.742318,392.912009,-980.730516,1276.069983,-839.970593,502.658138
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274395,75.591000,2.548039,4.502671,378.225376,-144.396932,398.418629,-128.513610,233.410200,3.494213,5.800339,617.864085,-234.706934,621.033964,-262.273805,-670.792531,-441.024069,-1250.602915,-873.604836,-392.736433,-25.188167
274419,213.901233,2.688396,3.512311,154.546734,-348.985621,198.128276,-335.539306,54.121333,2.270603,4.486622,-353.322400,-61.577304,-306.518826,-53.515864,630.350817,-287.177194,1516.309247,-1209.978064,138.754469,-851.981054
274451,74.326600,2.146982,2.908568,-63.866608,602.353514,-64.030529,501.371086,353.050600,3.374965,4.625484,471.111361,447.173915,446.714138,470.757912,-118.096167,148.520571,100.330098,1385.587907,-1109.903518,1116.898066
274492,180.656100,2.347310,4.101609,-366.843865,-4.492916,-358.220125,1.675258,310.347267,3.610400,5.867564,-711.350433,-49.085662,-657.150266,-47.833011,842.260443,-56.202457,1156.777073,-86.942085,1081.361979,-107.710022


,area_block,dist_block,dist_to_qb_block,x_force_block,y_force_block,norm_weighted_x_block,norm_weighted_y_block,area_rush,dist_rush,dist_to_qb_rush,x_force_rush,y_force_rush,norm_weighted_x_rush,norm_weighted_y_rush,net_x_force_bottom,net_y_force_bottom,net_x_force_middle,net_y_force_middle,net_x_force_top,net_y_force_top
242425,60.111667,2.436063,4.714007,-171.437763,-61.052623,-172.627751,-61.598698,238.906867,3.394794,5.681244,-271.766149,-67.273248,-265.571865,-65.515135,936.679110,-8.552174,1142.599590,-263.855576,1005.798104,15.742248
242460,244.908400,3.314509,4.402922,281.029668,20.122480,294.485725,-10.544926,133.005200,2.933624,4.907052,329.319698,2.046031,325.171175,8.569885,-1562.100556,-137.986278,-2340.844832,768.664330,-94.355835,-828.285231
242477,180.791000,2.552367,4.188961,334.613453,4.881392,338.808803,13.335453,406.929800,3.228710,4.962043,257.032511,32.053550,252.473751,28.709129,-1972.024282,843.390458,-1018.532176,-129.060764,-1098.856485,-253.517935
242509,0.000000,2.385366,4.387012,219.526445,-87.535538,234.102046,-83.168091,272.724600,3.664779,5.640069,305.234105,73.206537,290.437914,66.879870,-1238.520459,381.257262,-2927.303653,753.417098,-1338.061511,65.344177
242538,48.426900,2.671061,4.492848,234.824197,327.339453,235.915415,327.409754,277.756200,3.133006,5.242222,126.874573,51.900850,128.312829,46.297304,0.000000,0.000000,-2397.208580,1647.585806,-1194.479042,-450.888179
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274400,169.949800,2.582610,4.825753,283.086426,-197.555010,271.993298,-174.331939,286.988200,3.282657,5.639286,224.674916,-88.010529,218.988213,-90.414678,-1056.023511,-345.667502,-2008.057965,-837.451065,-828.650462,-172.879761
274424,207.489733,2.341787,4.197206,-96.141270,-293.743786,-47.699671,-297.301639,248.307900,2.271073,5.333106,-303.256906,0.162784,-318.038259,21.529208,842.746336,-452.939270,1668.874602,-2097.704414,783.225129,-367.933748
274456,77.780200,2.302151,4.380566,-1.089329,277.508452,-5.392441,206.964391,264.175500,3.738456,5.733859,283.551417,246.488907,288.440252,248.180870,0.000000,0.000000,-2083.970475,1215.164542,-1187.231729,1803.449187
274497,178.815000,2.596322,4.060846,-233.506496,-29.116137,-231.810673,-23.050857,28.223400,3.523478,5.193730,-478.216743,-26.318470,-469.073819,-30.211448,1020.911473,-5.417334,1313.965610,-4.941784,1199.942632,-334.100265


,area_block,dist_block,dist_to_qb_block,x_force_block,y_force_block,norm_weighted_x_block,norm_weighted_y_block,area_rush,dist_rush,dist_to_qb_rush,x_force_rush,y_force_rush,norm_weighted_x_rush,norm_weighted_y_rush,net_x_force_bottom,net_y_force_bottom,net_x_force_middle,net_y_force_middle,net_x_force_top,net_y_force_top
242431,470.074767,2.516331,4.335441,-247.499669,0.425144,-246.588236,-4.454149,407.781333,3.408583,5.081922,-461.894630,-64.697805,-456.586366,-66.015856,1426.882173,432.241165,1931.931514,-442.462207,1421.805009,-275.651800
242483,186.333900,2.250075,3.538741,484.544001,44.833590,504.183283,46.248219,181.832300,2.501681,4.101109,416.673251,59.160866,395.358662,86.290856,0.000000,0.000000,-2129.677925,1862.512310,-417.248476,-236.067899
242515,91.110800,2.340967,4.112816,520.657477,78.313289,513.081658,85.392043,312.862400,3.003738,4.737613,725.149574,202.112925,708.702226,239.184245,-482.518242,330.758398,-1846.103890,1381.435654,-1267.760520,-746.173866
242544,285.937300,2.650074,5.418950,427.355541,153.426575,420.356026,139.941249,362.264100,2.977680,6.107778,256.888637,69.034476,257.890985,59.340925,0.000000,0.000000,-2546.547638,635.503663,-741.413378,-695.061434
242587,224.679933,2.946495,4.238446,-318.146664,48.835981,-326.238414,44.131942,426.475900,4.042035,5.021884,-296.010763,52.963172,-300.279774,56.972762,1943.359506,673.321100,2436.136809,-127.841271,1133.192635,-30.725870
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274316,405.123900,2.841854,5.552637,280.546243,-62.479836,282.826088,-60.586695,683.686000,3.579642,6.432427,372.065099,-103.398168,372.918558,-116.523440,-441.635710,-315.464195,-533.137845,-1719.391919,-436.462558,395.764143
274375,119.714533,2.236768,4.254218,-256.542452,-46.873619,-249.564544,-51.987832,710.298067,2.939264,4.786471,-474.928889,-7.555524,-460.615189,-34.296292,670.993908,93.666255,828.832933,-108.443429,744.961560,-405.660367
274406,374.094000,2.371063,4.452863,397.124726,-126.838591,378.994513,-128.158586,415.336800,3.065624,5.142329,476.777076,-180.451361,466.855818,-167.591344,-716.788557,1191.182870,-3134.795749,-631.298824,-722.541292,-9.963129
274430,295.641867,2.303313,5.215694,-131.584057,-264.581477,-111.047632,-258.313117,266.541000,2.757967,6.352228,-448.434748,-160.385116,-451.362968,-152.243352,767.182109,-598.723625,3968.458195,-1729.945708,999.343502,-23.864600


...

,area_block,dist_block,dist_to_qb_block,x_force_block,y_force_block,norm_weighted_x_block,norm_weighted_y_block,area_rush,dist_rush,dist_to_qb_rush,x_force_rush,y_force_rush,norm_weighted_x_rush,norm_weighted_y_rush,net_x_force_bottom,net_y_force_bottom,net_x_force_middle,net_y_force_middle,net_x_force_top,net_y_force_top
242560,94.764100,2.019649,4.209079,149.485195,-16.922846,146.122937,-21.628628,148.157500,1.678010,4.719742,128.359232,32.224587,131.242776,30.352550,0.000000,0.000000,144.860310,557.434918,0.000000,0.000000
242924,47.824767,2.205457,2.419304,-123.540504,180.154542,-69.577071,327.246801,81.703000,1.968338,2.102281,-131.141559,157.804105,-0.045719,430.469455,0.000000,0.000000,-289.642407,485.346590,0.000000,0.000000
242973,815.022500,4.232013,12.148907,238.738504,-85.274995,292.853863,26.742343,170.654200,5.993067,10.539390,319.179263,142.614648,522.216579,289.987661,-345.493578,607.804123,-196.847605,2818.600290,0.000000,0.000000
243228,413.012000,4.638326,11.664661,195.611372,-288.217305,131.978621,-318.728533,423.240600,6.478536,10.029359,78.162225,-342.775775,-18.342331,-563.901547,546.136623,-1483.726384,-1532.624630,-2286.840477,0.000000,0.000000
243546,16.536400,2.750063,2.758423,-272.113736,93.638137,-289.934061,111.545644,130.150900,3.932743,3.959831,-438.602690,147.755803,-467.488068,197.947584,0.000000,0.000000,1500.347140,4350.967040,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273908,405.226600,2.855719,7.061102,450.320577,35.692493,476.500376,29.745380,468.461800,4.544912,7.682163,412.880990,67.696024,421.373196,79.006464,-940.077906,451.823173,-1297.610378,689.305294,-173.387694,-509.040292
274004,83.921533,3.705832,3.737850,2.098157,-62.096797,57.609067,-75.467738,111.547733,4.259752,4.263277,-122.050809,42.229689,-68.111423,52.819327,-910.281399,-198.555601,-1254.981525,-1026.109859,30.782233,-582.962769
274145,215.522533,4.822864,19.073574,-99.260754,-339.094793,-74.484223,-159.332942,1175.884133,6.360325,18.714283,355.779189,-124.181438,332.665186,67.659978,-1814.635128,-3379.332574,-271.011212,479.208004,0.000000,0.000000
274332,351.599000,4.099954,5.580684,126.384105,-388.818370,237.439961,-364.214047,118.940800,4.899790,6.609265,63.262780,-168.342153,222.706415,-109.187198,595.090992,1398.841404,189.684427,-570.544830,101.871418,-532.474400


,area_block,dist_block,dist_to_qb_block,x_force_block,y_force_block,norm_weighted_x_block,norm_weighted_y_block,area_rush,dist_rush,dist_to_qb_rush,x_force_rush,y_force_rush,norm_weighted_x_rush,norm_weighted_y_rush,net_x_force_bottom,net_y_force_bottom,net_x_force_middle,net_y_force_middle,net_x_force_top,net_y_force_top
242566,89.087200,1.935230,3.873291,8.889543,7.835282,27.050395,-2.825657,19.607800,1.558757,4.579036,-49.549392,127.605795,-33.166088,103.765484,0.000000,0.000000,152.619203,-317.883492,0.000000,0.000000
242930,211.671233,2.413594,3.241638,-80.104994,55.009842,148.078993,81.233219,50.991000,2.222130,2.473243,172.541871,52.574302,236.029692,320.129859,0.000000,0.000000,-317.890123,-5.408696,0.000000,0.000000
243234,380.100600,5.359378,13.839309,174.098903,-256.009392,98.783836,-255.421867,73.413100,7.664836,11.281471,19.332237,-415.086688,-106.808511,-440.908448,961.336824,-2980.222170,-832.262527,-2301.170603,0.000000,0.000000
243552,123.658667,3.100696,3.422216,-323.413926,384.899006,-415.666789,468.774592,140.214233,4.242536,4.677786,29.180634,606.617958,75.321557,712.250322,0.000000,0.000000,-1078.057632,2411.493932,-982.882293,328.104947
243722,160.022867,2.388246,4.624600,211.957219,-135.513970,249.759949,-155.206925,155.090267,1.968370,5.133762,-506.404464,-248.738834,-476.862642,-256.016970,539.231301,-234.464468,-1620.191640,201.539415,-168.896672,91.627171
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273693,148.396500,2.132557,2.228170,-67.383031,-95.242507,-51.662373,-110.450871,54.165933,0.704919,1.020084,199.290977,-51.247886,197.856926,-52.837182,0.000000,0.000000,-1558.891237,-2235.310326,0.000000,0.000000
273914,133.202700,2.780183,6.501188,153.038083,-7.876861,177.941490,-7.676053,371.078700,4.354322,6.860206,248.807932,113.224863,377.333863,137.589223,0.000000,0.000000,-3278.997796,3307.988430,-1987.787252,308.102702
274010,27.485800,4.018597,4.136498,54.920125,-47.605825,214.818876,-20.719843,110.850533,3.660185,3.666015,464.970002,-392.399794,405.574979,-321.929355,-832.627876,443.335640,-1971.779077,782.205369,835.543728,-767.870539
274338,681.623400,4.012699,6.040508,-81.378916,-117.223543,-13.580252,-134.439351,136.426700,4.610868,6.741731,-159.917359,293.979932,-49.053413,281.913504,1627.587569,2124.580071,540.390878,386.793109,0.000000,0.000000


,area_block,dist_block,dist_to_qb_block,x_force_block,y_force_block,norm_weighted_x_block,norm_weighted_y_block,area_rush,dist_rush,dist_to_qb_rush,x_force_rush,y_force_rush,norm_weighted_x_rush,norm_weighted_y_rush,net_x_force_bottom,net_y_force_bottom,net_x_force_middle,net_y_force_middle,net_x_force_top,net_y_force_top
242935,30.979667,2.639293,4.619441,38.105347,49.603543,80.460975,69.657423,37.642600,2.968951,3.373475,31.840868,-63.356674,47.233576,82.697575,0.000000,0.000000,-1706.799583,503.092305,0.000000,0.000000
243557,191.940267,3.617708,3.911316,279.805090,307.575825,442.344691,411.450675,182.233600,4.492336,4.812652,165.478693,300.429867,369.320439,331.210834,0.000000,0.000000,-922.546406,2412.293111,-260.963986,38.303170
243727,145.091667,2.753385,6.981948,129.720774,-120.702792,148.166901,-104.162027,0.000000,2.063227,7.379220,294.037323,756.985618,296.990176,757.507674,1126.671524,-187.272491,-2132.027249,1023.989276,-428.511237,331.549951
243815,40.093900,2.380929,8.422324,-493.256848,195.169045,-496.622836,241.705396,35.659600,1.308288,8.967875,-267.310597,28.230575,-226.219685,40.390857,862.330180,230.093130,1480.406210,722.815438,-48.425581,410.372656
243864,108.939533,2.036273,2.545245,19.795031,-446.680398,8.409870,-454.249917,43.884500,2.277191,3.668590,-52.957630,-176.914863,34.194574,-207.472370,-2346.773580,-109.935388,212.595552,804.272792,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272417,1318.740800,4.823719,5.467652,226.694257,-451.230060,509.491407,-694.710363,435.865300,5.140241,5.857831,-514.790978,80.866205,-932.215845,-118.850938,-475.381284,-1222.057693,559.421843,767.810815,-557.185293,856.032116
272579,1.586700,3.432696,6.037688,98.898844,-43.697525,224.186680,-14.031835,190.468800,4.552947,7.153954,450.658258,-446.606223,610.856036,-584.538534,-830.092146,-876.348523,-208.786297,-1066.128321,-1164.469509,-105.839654
272716,34.540533,2.785122,7.166624,45.714358,36.740089,-61.724256,25.265912,23.766967,3.375158,5.721509,-359.226252,-334.189232,-395.287134,-316.883825,0.000000,0.000000,659.671075,-1367.451197,-201.375235,-278.700488
272959,150.439200,3.128293,11.802365,35.204143,-108.989070,32.954603,-105.238378,87.588100,5.279169,9.394763,356.572456,-447.640267,818.217487,-762.037386,-694.695842,-1118.778332,30.451298,-1069.525747,-577.059463,-190.716341


,area_block,dist_block,dist_to_qb_block,x_force_block,y_force_block,norm_weighted_x_block,norm_weighted_y_block,area_rush,dist_rush,dist_to_qb_rush,x_force_rush,y_force_rush,norm_weighted_x_rush,norm_weighted_y_rush,net_x_force_bottom,net_y_force_bottom,net_x_force_middle,net_y_force_middle,net_x_force_top,net_y_force_top
243563,70.098967,3.953578,3.891201,93.772962,95.105454,215.048547,118.550592,149.395267,4.359635,4.328608,178.661402,493.767200,261.193399,329.500848,0.000000,0.000000,-2996.642808,1191.752517,-2231.930218,928.225650
243733,84.137933,3.201883,9.314667,193.875000,-78.746857,224.558127,-31.893323,246.180267,2.665305,9.692243,638.048276,504.692590,629.831986,498.093541,48.971081,-503.062051,-722.515978,2007.262349,-648.146125,577.157859
243870,212.406400,2.390195,3.903354,12.875526,53.685590,-23.655282,162.833341,9.944133,2.716970,5.237970,517.450179,106.477336,486.370244,124.555047,-1211.486524,581.256937,-690.546696,949.562154,0.000000,0.000000
243946,143.107800,2.574642,3.563170,-136.942933,-200.171712,-79.951868,-175.349895,11.644200,2.801877,4.159878,-134.026871,-271.833835,-65.005771,-215.028897,1652.710857,-723.494023,-1336.634304,228.843718,0.000000,0.000000
245344,196.143000,5.908054,15.386539,32.715471,153.928455,35.479036,208.041324,183.772267,8.463917,11.147546,166.633165,213.294655,31.432427,248.408358,-1606.803994,673.649700,-184.447383,113.783683,-95.590391,2292.204185
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272080,793.095600,4.970991,22.988352,-95.275359,298.134477,-97.007828,311.113094,459.264400,13.351683,17.784262,392.616723,833.498433,441.746360,1003.554153,-188.534386,647.166435,-87.637902,1145.239991,-953.056545,2526.951690
272423,1492.222500,5.111140,7.090844,-111.199324,-190.546953,-355.194093,-329.679877,512.924000,5.189342,6.897797,257.285444,338.629947,278.862056,313.885286,-255.220032,-439.121304,-939.319311,1009.851902,-1134.126527,242.218822
272585,289.033600,3.991834,7.964760,286.738790,-293.075813,370.733384,-389.619567,23.618100,5.230010,8.588708,192.413483,-145.734439,173.137613,-136.385243,-929.368937,-1853.720371,-723.087811,-2018.258206,-313.082163,-144.929494
272722,48.657267,3.122906,9.319465,33.374948,-55.524540,14.916832,-54.074330,258.557700,4.050411,7.347412,-250.858888,-335.887949,-243.315274,-388.816700,-104.734802,-394.361724,369.432723,866.900572,0.000000,0.000000


,area_block,dist_block,dist_to_qb_block,x_force_block,y_force_block,norm_weighted_x_block,norm_weighted_y_block,area_rush,dist_rush,dist_to_qb_rush,x_force_rush,y_force_rush,norm_weighted_x_rush,norm_weighted_y_rush,net_x_force_bottom,net_y_force_bottom,net_x_force_middle,net_y_force_middle,net_x_force_top,net_y_force_top
243738,101.192333,3.665425,11.352912,151.832785,25.349550,167.040243,65.619644,0.000000,3.572141,11.061737,286.067508,361.766945,281.452151,366.240060,-383.617387,131.715781,-553.737636,1491.977993,-778.165264,1370.732146
243875,117.464967,3.010897,5.867861,156.922789,117.780004,133.447474,203.607251,55.256733,3.347849,6.935283,279.354864,235.479769,250.703912,279.525492,-1400.994017,-329.983699,-867.612023,1771.459201,3.451052,552.297205
243951,179.972400,3.020247,5.852239,17.577636,-156.507975,73.571420,-147.331381,136.707200,3.236304,5.758207,-105.385621,111.099341,-62.099575,201.741346,2581.560965,38.526205,-421.769962,1706.207703,0.000000,0.000000
245349,180.948033,6.560354,16.053460,197.567543,210.723140,154.365807,276.405126,163.793233,8.813377,10.506467,299.668120,675.340530,-64.634624,899.285777,-710.028140,777.133288,-4.236036,189.582685,776.324538,2503.251313
245637,421.809600,3.002459,5.257126,-320.429468,-102.937987,-342.297538,-139.985766,530.722100,1.817635,4.295814,-607.394649,-160.235871,-497.303362,-232.878438,1224.175055,-1199.905923,771.843982,-671.680717,597.652483,-319.297475
247753,247.475400,5.245262,9.262150,-72.976243,0.745003,25.826849,58.715097,104.963800,5.988785,9.011333,198.038340,77.141875,430.486344,103.049573,1167.332300,1415.697918,122.576385,179.488303,-1256.656470,132.227155
248932,116.356467,3.357578,7.217181,-174.693575,43.176466,-307.819148,52.262129,437.791233,3.989878,4.763567,-286.472765,-245.353855,-530.775640,-906.097911,458.327129,38.003488,2124.936384,-2329.656325,1023.473652,-60.932246
249584,373.479000,3.964298,7.687398,60.721778,-311.338710,37.267207,-336.077522,274.819033,3.000912,2.828812,-85.794398,-380.053495,-232.655082,-432.640009,374.056054,-3269.479318,-104.915475,-113.496867,0.000000,0.000000
250423,150.922933,2.999711,7.162568,-187.118283,181.728785,-191.610831,195.706805,170.648067,2.765781,3.665263,-671.939160,255.148424,-789.609189,222.137309,1465.164550,728.612537,3136.455275,-1066.680607,0.000000,0.000000
251558,369.827500,4.826009,10.651536,19.740427,233.108484,5.313024,218.499853,254.771833,4.539939,8.070392,62.807901,376.632839,46.985225,370.285096,0.000000,0.000000,-290.260505,743.561026,-105.699827,2217.017117


In [ ]:
print(len(models))
print(len(test_by_frame_list))

In [7]:
# actually do the evaluation
from sklearn.metrics import accuracy_score, recall_score
for i, model in enumerate(models):
    (x_test, y_test) = test_by_frame_list[i]
    predicted = models[i].predict(x_test)
    accuracy = accuracy_score(y_test, predicted)
    recall = recall_score(y_test, predicted)
    
    
    print(f'Out-of-bag score estimate: {models[i].oob_score_:.3}')
    print(f'Mean accuracy score: {accuracy:.3}')
    print(f'Recall: {recall:.3}')
    

Out-of-bag score estimate: 0.621
Mean accuracy score: 0.622
Recall: 0.036
Out-of-bag score estimate: 0.623
Mean accuracy score: 0.623
Recall: 0.0643
Out-of-bag score estimate: 0.621
Mean accuracy score: 0.618
Recall: 0.0541
Out-of-bag score estimate: 0.626
Mean accuracy score: 0.634
Recall: 0.0803
Out-of-bag score estimate: 0.619
Mean accuracy score: 0.596
Recall: 0.167
Out-of-bag score estimate: 0.616
Mean accuracy score: 0.622
Recall: 0.457
Out-of-bag score estimate: 0.656
Mean accuracy score: 0.664
Recall: 0.711
Out-of-bag score estimate: 0.682
Mean accuracy score: 0.722
Recall: 0.909
Out-of-bag score estimate: 0.735
Mean accuracy score: 0.819
Recall: 0.955
Out-of-bag score estimate: 0.782
Mean accuracy score: 0.841
Recall: 0.967
Out-of-bag score estimate: 0.813
Mean accuracy score: 0.885
Recall: 0.982
Out-of-bag score estimate: 0.791
Mean accuracy score: 0.829
Recall: 0.967
